In [6]:
from flask import Flask, request, jsonify
import psycopg2
import os
from typing import List, Dict, Union

app = Flask(__name__)

# Veritabanı bağlantısı
def get_db_connection():
    conn = psycopg2.connect(
    dbname="smartcity",
    user="postgres",
    password="123asd",
    host="localhost",
    port="5432"
)
    return conn
@app.route('/')
def home():
    return "<h1>Emlak API</h1><p>Batch ekleme: POST /properties/batch</p>"

# Tek ilan ekleme endpoint'i
@app.route('/properties', methods=['POST'])
def add_property():
    data = request.get_json()
    
    required_fields = [
        'city', 'neighborhood', 'building_age', 'totalrooms',
        'bathroom_count', 'luxury_score', 'price_per_m2',
        'room_density', 'location_tier', 'm2_location_interact'
    ]
    
    if not all(field in data for field in required_fields):
        return jsonify({"error": "Missing required fields"}), 400

    try:
        conn = get_db_connection()
        cur = conn.cursor()
        
        cur.execute("""
            INSERT INTO property_data (
                city, neighborhood, building_age, totalrooms, bathroom_count,
                luxury_score, furnished, air_conditioning, has_balcony,
                price_per_m2, room_density, location_tier, m2_location_interact
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            RETURNING id
        """, (
            data['city'], data['neighborhood'], data['building_age'],
            data['totalrooms'], data['bathroom_count'], data['luxury_score'],
            data.get('furnished', False), data.get('air_conditioning', False),
            data.get('has_balcony', False), data['price_per_m2'],
            data['room_density'], data['location_tier'], data['m2_location_interact']
        ))
        
        property_id = cur.fetchone()[0]
        conn.commit()
        
        return jsonify({
            "id": property_id,
            "status": "created",
            "data": data
        }), 201
        
    except Exception as e:
        conn.rollback()
        return jsonify({"error": str(e)}), 500
        
    finally:
        cur.close()
        conn.close()

# Toplu ilan ekleme endpoint'i
@app.route('/properties/batch', methods=['POST'])
def add_properties_batch():
    properties = request.get_json()
    
    if not isinstance(properties, list):
        return jsonify({"error": "Expected a list of properties"}), 400

    results = []
    errors = []
    
    conn = get_db_connection()
    cur = conn.cursor()
    
    try:
        for idx, prop in enumerate(properties):
            try:
                cur.execute("""
                    INSERT INTO property_data (
                        city, neighborhood, building_age, totalrooms, bathroom_count,
                        luxury_score, furnished, air_conditioning, has_balcony,
                        price_per_m2, room_density, location_tier, m2_location_interact
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    RETURNING id
                """, (
                    prop['city'], prop['neighborhood'], prop['building_age'],
                    prop['totalrooms'], prop['bathroom_count'], prop['luxury_score'],
                    prop.get('furnished', False), prop.get('air_conditioning', False),
                    prop.get('has_balcony', False), prop['price_per_m2'],
                    prop['room_density'], prop['location_tier'], prop['m2_location_interact']
                ))
                
                property_id = cur.fetchone()[0]
                results.append({
                    "index": idx,
                    "id": property_id,
                    "status": "success"
                })
                
            except Exception as e:
                errors.append({
                    "index": idx,
                    "error": str(e),
                    "data": prop
                })
        
        conn.commit()
        
        return jsonify({
            "success_count": len(results),
            "failed_count": len(errors),
            "results": results,
            "errors": errors
        }), 207  # Multi-Status
        
    except Exception as e:
        conn.rollback()
        return jsonify({"error": str(e)}), 500
        
    finally:
        cur.close()
        conn.close()
def run_flask():
    app.run(port=8001)  # Farklı bir port kullanın


In [7]:
# Flask sunucusunu başlat (zaten yapmışsınız)
import threading
thread = threading.Thread(target=run_flask)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 8001 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [5]:
import requests

# Örnek veri
sample_data = [
    {
        "city": "Istanbul",
        "neighborhood": "Kadikoy",
        "building_age": 5,
        "totalrooms": 3,
        "bathroom_count": 2,
        "luxury_score": 8.5,
        "furnished": True,
        "air_conditioning": True,
        "has_balcony": True,
        "price_per_m2": 15000.0,
        "room_density": 0.2,
        "location_tier": 1,
        "m2_location_interact": 47.805547
    }
]

# İstek gönderme
response = requests.post("http://localhost:8001/properties/batch", json=sample_data)
print(response.json())

{'errors': [], 'failed_count': 0, 'results': [{'id': 3, 'index': 0, 'status': 'success'}], 'success_count': 1}


In [15]:
import random
import requests
from faker import Faker

fake = Faker('tr_TR')  # Türkçe veri üretimi için

def generate_sample_property():
    return {
        "city": random.choice(["Istanbul"]),
        "neighborhood": fake.street_name(),
        "building_age": random.randint(1, 30),
        "totalrooms": random.randint(1, 5),
        "bathroom_count": random.randint(1, 3),
        "luxury_score": round(random.uniform(5.0, 9.9), 1),
        "furnished": random.choice([True, False]),
        "air_conditioning": random.choice([True, False]),
        "has_balcony": random.choice([True, False]),
        "price_per_m2": random.randint(5000, 45000),
        "room_density": round(random.uniform(1, 5), 2),
        "location_tier": random.randint(1, 4),
        "m2_location_interact": round(random.uniform(20, 80), 6)
    }

# 100 örnek ilan oluştur
sample_properties = [generate_sample_property() for _ in range(400)]

In [16]:
# API endpoint'iniz
API_URL = "http://localhost:8001/properties/batch"

# 100 ilanı 10'arlı gruplar halinde gönder (daha stabil çalışır)
batch_size = 10
for i in range(0, len(sample_properties), batch_size):
    batch = sample_properties[i:i + batch_size]
    response = requests.post(API_URL, json=batch)
    
    print(f"Batch {i//batch_size + 1} Durumu:")
    print(f"HTTP Kodu: {response.status_code}")
    print(response.json())
    print("-" * 50)
    

Batch 1 Durumu:
HTTP Kodu: 207
{'errors': [], 'failed_count': 0, 'results': [{'id': 304, 'index': 0, 'status': 'success'}, {'id': 305, 'index': 1, 'status': 'success'}, {'id': 306, 'index': 2, 'status': 'success'}, {'id': 307, 'index': 3, 'status': 'success'}, {'id': 308, 'index': 4, 'status': 'success'}, {'id': 309, 'index': 5, 'status': 'success'}, {'id': 310, 'index': 6, 'status': 'success'}, {'id': 311, 'index': 7, 'status': 'success'}, {'id': 312, 'index': 8, 'status': 'success'}, {'id': 313, 'index': 9, 'status': 'success'}], 'success_count': 10}
--------------------------------------------------
Batch 2 Durumu:
HTTP Kodu: 207
{'errors': [], 'failed_count': 0, 'results': [{'id': 314, 'index': 0, 'status': 'success'}, {'id': 315, 'index': 1, 'status': 'success'}, {'id': 316, 'index': 2, 'status': 'success'}, {'id': 317, 'index': 3, 'status': 'success'}, {'id': 318, 'index': 4, 'status': 'success'}, {'id': 319, 'index': 5, 'status': 'success'}, {'id': 320, 'index': 6, 'status': 'su

In [17]:
# Veritabanından sayım yapma
def count_properties():
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM property_data")
    count = cur.fetchone()[0]
    conn.close()
    return count

print(f"Toplam ilan sayısı: {count_properties()}")

Toplam ilan sayısı: 703


In [14]:
conn=get_db_connection()
cur=conn.cursor()
cur.execute("SELECT * FROM property_data LIMIT 10")
rows = cur.fetchall()
for row in rows:
    print(row)

cur.close()
conn.close()

(1, 'Istanbul', 'Kadikoy', 15, 3, 2, 8.5, True, False, True, 12000.0, 0.2, 1, 0.0)
(2, 'Istanbul', 'Kadikoy', 15, 3, 2, 8.5, True, False, True, 12000.0, 0.2, 1, 0.0)
(3, 'Istanbul', 'Kadikoy', 5, 3, 2, 8.5, True, True, True, 15000.0, 0.2, 1, 47.805547)
(4, 'Izmir', 'Erdoğan Pine', 26, 1, 1, 9.3, True, True, False, 32583.0, 1.04, 3, 25.584983)
(5, 'Izmir', 'Oğurata Turnpike', 24, 1, 1, 5.3, False, False, True, 24724.0, 1.58, 1, 78.564123)
(6, 'Istanbul', 'Gencaslan Square', 10, 1, 2, 9.2, True, True, True, 37316.0, 4.49, 4, 60.249144)
(7, 'Istanbul', 'Yaman Inlet', 19, 3, 2, 5.3, True, True, True, 40347.0, 4.65, 1, 49.205928)
(8, 'Izmir', 'Ülker Highway', 19, 3, 3, 6.8, True, True, True, 31467.0, 4.96, 3, 42.797555)
(9, 'Istanbul', 'Perinur Mountain', 12, 5, 3, 8.7, False, True, True, 23846.0, 4.32, 2, 20.63)
(10, 'Izmir', 'Ağmur Coves', 3, 3, 3, 5.3, False, False, True, 27627.0, 2.31, 2, 61.999642)
